<h1>Importando bibliotecas</h1>

In [182]:
import pandas as pd
import numpy as np
import sys
import sklearn
from scipy.spatial import distance
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.metrics import log_loss
import _pickle as cPickle
from gekko import GEKKO

<h1>Importando dados</h1>

In [5]:
#importando amostras normais classificadas como ataque
dfDoS = pd.read_csv('datasets_adv/DoS_orig_normal.csv')
dfDoS

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i,y_actual,y_pred
0,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
1,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
6,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
7,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
8,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1


In [8]:
#importando o normalizer e o classificador
with open('classificadores/rfc_DoS.pkl', 'rb') as fid:
    rfc_DoS = cPickle.load(fid)  
with open('scalers/scalerDoS.pkl', 'rb') as fid:
    scalerDoS = cPickle.load(fid)  

In [12]:
dfDoS = dfDoS.drop('y_actual', 1)
dfDoS = dfDoS.drop('y_pred', 1)
dfDoS

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_http_8001,service_http_2784,service_urh_i,service_aol,service_harvest,service_red_i
0,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1480.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,28.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
#variáveis inteiras
col_namesInt = ['land',  'logged_in',  'is_host_login', 'is_guest_login',]

#variáveis contínuas
col_namesFloat = ['duration',
 'src_bytes',
 'dst_bytes',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
]
#variáveis categoricas => one_hot_encoder => variáveis inteiras
categ = ['protocol_type_icmp',
 'protocol_type_tcp',
 'protocol_type_udp', 'service_IRC',
 'service_X11',
 'service_Z39_50',
 'service_aol',
 'service_auth',
 'service_bgp',
 'service_courier',
 'service_csnet_ns',
 'service_ctf',
 'service_daytime',
 'service_discard',
 'service_domain',
 'service_domain_u',
 'service_echo',
 'service_eco_i',
 'service_ecr_i',
 'service_efs',
 'service_exec',
 'service_finger',
 'service_ftp',
 'service_ftp_data',
 'service_gopher',
 'service_harvest',
 'service_hostnames',
 'service_http',
 'service_http_2784',
 'service_http_443',
 'service_http_8001',
 'service_imap4',
 'service_iso_tsap',
 'service_klogin',
 'service_kshell',
 'service_ldap',
 'service_link',
 'service_login',
 'service_mtp',
 'service_name',
 'service_netbios_dgm',
 'service_netbios_ns',
 'service_netbios_ssn',
 'service_netstat',
 'service_nnsp',
 'service_nntp',
 'service_ntp_u',
 'service_other',
 'service_pm_dump',
 'service_pop_2',
 'service_pop_3',
 'service_printer',
 'service_private',
 'service_red_i',
 'service_remote_job',
 'service_rje',
 'service_shell',
 'service_smtp',
 'service_sql_net',
 'service_ssh',
 'service_sunrpc',
 'service_supdup',
 'service_systat',
 'service_telnet',
 'service_tftp_u',
 'service_tim_i',
 'service_time',
 'service_urh_i',
 'service_urp_i',
 'service_uucp',
 'service_uucp_path',
 'service_vmnet',
 'service_whois','flag_OTH',
 'flag_REJ',
 'flag_RSTO',
 'flag_RSTOS0',
 'flag_RSTR',
 'flag_S0',
 'flag_S1',
 'flag_S2',
 'flag_S3',
 'flag_SF',
 'flag_SH']

In [76]:
minimos = dfDoS.min()
maximos = dfDoS.max()

In [90]:
tipoInteiro = {}

In [91]:
for s in col_namesInt:
    tipoInteiro[s] = True
for s in col_namesFloat:
    tipoInteiro[s] = False
for s in categ:
    tipoInteiro[s] = True
tipoInteiro

{'land': True,
 'logged_in': True,
 'is_host_login': True,
 'is_guest_login': True,
 'duration': False,
 'src_bytes': False,
 'dst_bytes': False,
 'wrong_fragment': False,
 'urgent': False,
 'hot': False,
 'num_failed_logins': False,
 'num_compromised': False,
 'root_shell': False,
 'su_attempted': False,
 'num_root': False,
 'num_file_creations': False,
 'num_shells': False,
 'num_access_files': False,
 'num_outbound_cmds': False,
 'count': False,
 'srv_count': False,
 'serror_rate': False,
 'srv_serror_rate': False,
 'rerror_rate': False,
 'srv_rerror_rate': False,
 'same_srv_rate': False,
 'diff_srv_rate': False,
 'srv_diff_host_rate': False,
 'dst_host_count': False,
 'dst_host_srv_count': False,
 'dst_host_same_srv_rate': False,
 'dst_host_diff_srv_rate': False,
 'dst_host_same_src_port_rate': False,
 'dst_host_srv_diff_host_rate': False,
 'dst_host_serror_rate': False,
 'dst_host_srv_serror_rate': False,
 'dst_host_rerror_rate': False,
 'dst_host_srv_rerror_rate': False,
 'protoc

<h2>Funções auxiliares</h2>

In [167]:
def gekko2list(varX):
    l = []
    for x in varX:
        l.append(x.value.value)
    return l

In [200]:
def gekkof2list(varX):
    l = []
    for x in varX:
        l.append(x[0])
    return l

In [201]:
def logloss(true_label, predicted, eps=1e-15):
 p = np.clip(predicted, eps, 1 - eps)
 if true_label == 1:
   return -np.log(p)
 else:
   return -np.log(1 - p)

In [369]:
#função objetivo provisória apenas para testar
def funcaoObjetivo(model, scaler, X0, X, y0):
    #normalizando dados
    Xi = np.array(X0).reshape(1, -1)
    Xi = scaler.transform(Xi)
    X2 = gekko2list(X)
    X2 = np.array(X2).reshape(1, -1)
    X2 = scaler.transform(X2)
    
    #usando o classificador (probabilidade de ser ataque)
    y = model.predict_proba(X2)[0][1]
    
    #retorna entropia cruzada entre (y0, y) + distância entre X e X0
    return logloss(y0, y) + distance.euclidean(X2, Xi)
    '''dist = 0
    for i in range(len(X)):
        dist = dist + (X[i] - X0[i])**2
    return -dist'''

<h1>Otimização</h1>

In [376]:
m = GEKKO() 
i = 2 #linha i do dataframe
#variáveis
X = []
X0 = []
#instanciando variáveis X (amostra adversária) e ponto inicial X0 (amostra original)
for c in dfDoS.columns:
    X0.append(dfDoS[c][i])
    if tipoInteiro[c]:
        #X.append(m.Var(value = int(dfDoS[c][i]), lb=int(minimos[c]), ub=int(maximos[c]), integer=True))
        X.append(m.Var(value =dfDoS[c][i], lb=dfDoS[c][i], ub=dfDoS[c][i], integer=True)) #travando as variáveis inteiras
    else:
        X.append(m.Var(value = dfDoS[c][i], lb=minimos[c], ub=maximos[c]))

In [377]:
#Objetivo: minimizar a função objetivo
m.Obj(funcaoObjetivo(rfc_DoS, scalerDoS, X0, X, 0))
m.solve(debug=True)

apm 189.35.246.92_gk_model37 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          122
   Intermediates:            0
   Connections  :            0
   Equations    :            1
   Residuals    :            1
 
 ________________________________________________
 ss.Eqn(1)
 0 = 0.8209805520698303
 ________________________________________________
 Number of state variables:            122
 Number of total equations: -            0
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            122
 
 Number of bound variables: -          101
 
 **********************************************
 Steady State Optimization with Interior Poi

In [378]:
#amostra adversária
X

[[0.0],
 [1247.004633],
 [0.0],
 [0.0],
 [1.1403121806],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [9.3482070041],
 [4.3440919287],
 [0.017150694816],
 [0.061902081642],
 [0.15457824518],
 [0.11038491535],
 [0.84969836032],
 [0.13176934181],
 [0.086146758635],
 [36.261975033],
 [22.334500809],
 [0.84399622508],
 [0.14174834718],
 [0.84399622508],
 [0.41385324137],
 [0.061902081642],
 [0.061902081642],
 [0.14032278666],
 [0.11038491535],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [8.6736173799e-19],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [8.6736173799e-19],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [-3.469446952e-18],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 

In [379]:
X0

[0.0,
 1480.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 2.0,
 0.0,
 0.0,
 0.0,
 -1.3877787807814454e-17,
 1.0,
 0.0,
 0.0,
 2.0,
 26.0,
 1.0,
 0.0,
 1.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.673617379884034e-19,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.673617379884034e-19,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -3.4694469519536134e-18,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.3881317890172006e-21,
 0.0,
 0.0,
 1.3552527156068802e-20,
 8.673617379884034e-19,
 0.0,
 8.673617379884034e-19,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [380]:
#aplicando o classificador para a amostra adversária
Xaux = gekkof2list(X)
Xaux = np.array(Xaux).reshape(1, -1)
rfc_DoS.predict_proba(scalerDoS.transform(Xaux))
#[probabilidade de ser amostra normal, prob de ser ataque]

array([[0.71, 0.29]])

In [381]:
#diferença entre as duas amostras
distance.euclidean(Xaux, X0)

235.65626189249866

In [389]:
max(max(abs(Xaux - X0)))

232.995367

<h2>Tentando otimizar com as variáveis inteiras</h2>
Ta dificil de rolar...

In [454]:
m2 = GEKKO()
m2.options.SOLVER = 1
m2.solver_options = ['minlp_maximum_iterations 50000', \
                    # minlp iterations with integer solution
                    'minlp_max_iter_with_int_sol 1000', \
                    # treat minlp as nlp
                    'minlp_as_nlp 0', \
                    # nlp sub-problem max iterations
                    'nlp_maximum_iterations 500', \
                    # 1 = depth first, 2 = breadth first
                    'minlp_branch_method 1', \
                    # maximum deviation from whole number
                    'minlp_integer_tol 0.00005', \
                    # covergence tolerance
                    'minlp_gap_tol 0.00001']

In [455]:
i = 0
#variáveis
X = []
X0 = []
for c in dfDoS.columns:
    X0.append(dfDoS[c][i])
    if tipoInteiro[c]:
        X.append(m2.Var(value = int(dfDoS[c][i]), lb=int(minimos[c]), ub=int(maximos[c]), integer=True))
        #X.append(m2.Var(value =dfDoS[c][i], lb=dfDoS[c][i], ub=dfDoS[c][i], integer=True)) #travando as variáveis inteiras
    else:
        X.append(m2.Var(value = dfDoS[c][i]+0.1, lb=minimos[c], ub=maximos[c]))

In [456]:
def distA(X, X0):
    dist = (X[1] - X0[1])**2
    #for i in range(len(X0)-1):
    #    dist = dist + (X[i+1] - X0[i+1])**2
    return dist

In [459]:
m2.Obj(-((X[1] - X0[1])**2)+5)

In [460]:
m2.solve()

apm 189.35.246.92_gk_model47 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          122
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:            122
 Number of total equations: -            0
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            122
 
 Number of bound variables: -           94
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I: -9 Tm:      0.05 NLPi:  501 Dpth:    0 Lvs:    0 Obj:  0.00E+00 Gap:   

Exception:  @error: Solution Not Found


In [ ]:
X